In [1]:
%matplotlib inline
import nibabel as nib
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [2]:
home_path = "C:/Users/User/msc_project/sus-optimization"

In [14]:
!git clone https://github.com/shimming-toolbox/susceptibility-to-fieldmap-fft.git
%cd susceptibility-to-fieldmap-fft
!git checkout mb/custom_pad
%pip install .
%cd {home_path}

c:\Users\User\msc_project\sus-optimization\susceptibility-to-fieldmap-fft\susceptibility-to-fieldmap-fft\susceptibility-to-fieldmap-fft


Cloning into 'susceptibility-to-fieldmap-fft'...
c:\Users\User\miniconda3\envs\mrsim\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


branch 'mb/custom_pad' set up to track 'origin/mb/custom_pad'.


Switched to a new branch 'mb/custom_pad'


Processing c:\users\user\msc_project\sus-optimization\susceptibility-to-fieldmap-fft\susceptibility-to-fieldmap-fft\susceptibility-to-fieldmap-fft
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for susceptibility-to-fieldmap-fft: filename=susceptibility_to_fieldmap_fft-0.0.0-py3-none-any.whl size=23545 sha256=4a1f4ad7ce00d61cba5796bf1ce4baf5ca2d17e4cae1145edd23226c9ea4efdc
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b9\bc\3f\dc236ce32c3c7e6735cc662c8aee149412acd5cdf33fa7ef4b
Successfully built susceptibility-to-fieldmap-fft
  Attempting uninstall: susceptibility-to-fieldmap-fft
    Found existing installation: susceptibility-to-fieldmap-fft 0.0.0
    Uninstalling susceptibility-to-fieldmap-fft-0.0.0:
      Successfully uninstalled susceptibility-to-fieldmap-fft-0.0.0
Note: you may need to restart the kernel to use updated packages.
C:\Users\User\msc_project\sus-optimization


In [3]:
!git clone https://github.com/shimming-toolbox/tissue-to-MRproperty.git
%cd tissue_to_MRproperty
!git checkout dev/fine-tune
%pip install .
%cd {home_path}

Note: you may need to restart the kernel to use updated packages.


ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [14]:
!tissue_to_mr --help

Usage: tissue_to_MR [OPTIONS] INPUT_FILE [OUTPUT_FILE]

Options:
  -s, --segtool [TotalSeg_CT|TotalSeg_MRI|ProCord_MRI|charles|compare_fm]
                                  State what segmentator was used  [required]
  -v, --version [v1|v2|mod0|mod1|mod2|dyn|mod_PAM50]
                                  Select the version of your segmentation file
                                  [required]
  -t, --type [t2s|sus|pd|t1|t2]   Please choose MR property to convert to
                                  [required]
  -g, --gauss [0|1]               Set to 1 to use Gaussian distribution
  -x, --chi FLOAT                 Used to define new chi value for FM
                                  comparison approach
  --help                          Show this message and exit.


# Sebastian Code 

In [18]:
import subprocess

def seg_to_chi(seg_file,chi,output_file):

    # Using our command to generate new chi map

    command = [
    "tissue_to_mr",
    seg_file,
    "-t", "sus",
    "-s", "compare_fm",
    "-v", "dyn",
    "-x", chi,
    output_file
]

    # Run the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)

    # Check for errors
    if result.returncode != 0:
        print("Error occurred:", result.stderr)
    else:
        print("Metrics extracted successfully:", result.stdout)

def chi_to_fm_ppm(input_file, output_file):

    #Using a custom command for dB0 simulation

    command = [
    "compute_fieldmap",
    "-i", input_file,
    "-o", output_file,
    "-b", "50",
    "-m", "b0SimISMRM"
    ]

    # Run the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print("Error occurred:", result.stderr)
    else:
        print("Metrics extracted successfully:", result.stdout)


In [6]:
# File location variable designation:
# Suggestion: put the final files in the data folder under this repository 
path_chimap = 'data/db0_030_wb_chimap.nii.gz'
path_labels = 'data/db0_030_segmentations.nii.gz'
path_fm_Hz = 'data/demod_ISMRM_sim_fm_Hz.nii.gz'

In [6]:
chimap_img = nib.load(path_chimap)

In [7]:
# Testing seg to chi converter
a = seg_to_chi(chimap_img,0.65,"output/0.65_chimap.nii.gz")

TypeError: expected str, bytes or os.PathLike object, not Nifti1Image

In [19]:
# Testing chi to FM
b = chi_to_fm_ppm(path_chimap,"output/testing_fbf_function.nii.gz")

Metrics extracted successfully: Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 29.95 seconds

